<a href="https://colab.research.google.com/github/sudhirsrajput/Misc/blob/main/text_summarization_main_notebooks_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/text-summarization-main_notebooks_news_summary.csv', encoding='latin-1', engine='python')

# New Section

In [ ]:
len(df)

4514

In [ ]:
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df = df[['text', 'ctext']]
df.columns = ['summary', 'text']
df.dropna()
df.head()

,summary,text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
df['text'][0]

'The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the Dama

In [ ]:
len(df['text'][1])

2382

In [ ]:
len(df['summary'][1])

361

In [ ]:
!pip install -q pytorch_lightning
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 32.1 MB/s eta 0:00:00


In [ ]:
import torch
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader
import re
from transformers import AdamW
from sklearn.model_selection import train_test_split

In [ ]:
## , this class is structured to facilitate the training of a model on source-target pairs of texts for summarization tasks, by providing a standardized way to preprocess and encode the data for model input.

class NewsDataset(Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, source_len, target_len):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.target_len = target_len

    def __len__(self):
        return len(self.target_texts) - 1

    def __getitem__(self, idx):
        ## whitespace_handler is sued to remove whitespcte after \s and \n
        whitespace_handler = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

        text = " ".join(str(self.source_texts[idx]).split()) ##" " helps to remove the space the the begining or end of the line
        summary = " ".join(str(self.target_texts[idx]).split())

        source = self.tokenizer.batch_encode_plus([whitespace_handler(text)],
                                                max_length= self.source_len,
                                                padding='max_length', ## to add the padding for each sentence
                                                truncation=True,
                                                return_attention_mask=True,  ## It will retrun the return_attention_mask
                                                add_special_tokens=True, ## add_special_tokens like for ? will be added
                                                return_tensors='pt')

        target = self.tokenizer.batch_encode_plus([whitespace_handler(summary)],
                                                max_length = self.target_len,
                                                padding='max_length',
                                                truncation=True,
                                                return_attention_mask=True,
                                                add_special_tokens=True,
                                                return_tensors='pt')

        labels = target['input_ids']
        labels[labels == 0] = -100 ##AS attension mask will add zero at the end, hence for label=0 it will assign -100.

        return (source['input_ids'].squeeze(), ##squeeze() will remove 1 dimesions from the output
                source['attention_mask'].squeeze(),
                labels.squeeze(),
                target['attention_mask'].squeeze())


# The batch_encode_plus method is used to encode a batch of text inputs for models in the Hugging Face Transformers library. Here's what it does:
# Tokenization: It converts the input text (in this case, summary) into tokens based on the model's vocabulary.
# Padding: If padding='max_length' is specified, it pads the sequences to a specified maximum length (max_length), ensuring all sequences in the batch have the same length.
# Truncation: If the input text exceeds the max_length, it truncates the sequences to fit.
# Attention Masks: It generates attention masks that indicate which tokens are actual tokens (1) and which are padding tokens (0). This is important for the model to focus only on the relevant tokens.
# Special Tokens: If add_special_tokens=True, it adds any necessary special tokens that the model requires (e.g., start and end tokens).
# Return Tensors: By specifying return_tensors='pt', it returns the encoded data as PyTorch tensors, which can be directly used in PyTorch-based models.


1. hi how are you -> 4

2. hi my name i ramesh -> 5

padding adding 0- at the last of each sentences

attention mask(0, 1)

1- (1,1,1,1,0)

2- (1,1,1,1,1)



In [ ]:
class NewsDataLoader(pl.LightningDataModule): ## pl.LightningDataModule is parent class
    def __init__(self, file_path, tokenizer, batch_size, val_split_size,
                 columns_name, source_len=1024, target_len=128, corpus_size=1000):
        super().__init__()
        self.tokenizer = tokenizer
        self.file_path = file_path
        self.batch_size = batch_size
        self.split_size = val_split_size
        self.nrows = corpus_size
        self.columns_name = columns_name
        self.target_len = target_len
        self.source_len = source_len

    def prepare_data(self):
        data = pd.read_csv(self.file_path, nrows=self.nrows, encoding='latin-1')
        data = data[self.columns_name]
        data = data.dropna()
        self.target_text = data.iloc[:,0].values
        self.source_text = data.iloc[:,-1].values

    def setup(self, stage=None):
        X_train, y_train, X_val, y_val = train_test_split(
            self.source_text, self.target_text, test_size=self.split_size
        )

        self.train_dataset = (X_train, y_train)
        self.val_dataset = (X_val, y_val)

    def train_dataloader(self):
        train_data = NewsDataset(source_texts=self.train_dataset[0],
                             target_texts=self.train_dataset[1],
                             tokenizer=self.tokenizer,
                             source_len=self.source_len,
                             target_len=self.target_len
                            )
        return DataLoader(train_data, self.batch_size, num_workers=6, shuffle=True, pin_memory=True)

    def val_dataloader(self):
        val_data = NewsDataset(source_texts=self.val_dataset[0],
                          target_texts=self.val_dataset[1],
                          tokenizer=self.tokenizer,
                          source_len=self.source_len,
                          target_len=self.target_len
                          )
        return DataLoader(val_data, self.batch_size, num_workers=6, pin_memory=True)

In [ ]:
class T5smallFinetuner(pl.LightningModule):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer

    def forward(self, input_ids, attention_mask,
                decoder_attention_mask=None, labels=None):

        outputs= self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )
        return outputs.loss

    def _step(self, batch):

        source_input_ids, source_attention_mask, target_input_ids, target_attention_mask = batch

        loss = self(input_ids=source_input_ids,
                      attention_mask=source_attention_mask,
                      decoder_attention_mask=target_attention_mask,
                      labels=target_input_ids
                      )
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def training_epoch_end(self, outputs):
        batch_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log('train_loss', batch_loss, prog_bar=True, logger=True)

    def validation_epoch_end(self, outputs):
        batch_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log('val_loss', batch_loss, prog_bar=True, logger=True)

    def configure_optimizers(self):
        model = self.model
        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
        return {
           'optimizer': optimizer,
           'lr_scheduler': scheduler,
           'monitor': 'val_loss'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
dataloader = NewsDataLoader(tokenizer=tokenizer,
                            file_path='news_summary.csv',
                            val_split_size=0.3, batch_size=4, columns_name=['text', 'ctext'])

dataloader.prepare_data()

dataloader.setup()

In [ ]:
for i in dataloader.train_dataloader():
  print(type(i), len(i), type(i[0]), i[2].shape)
  break

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


<class 'list'> 4 <class 'torch.Tensor'> torch.Size([4, 128])


In [ ]:
model = T5smallFinetuner(model, tokenizer)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)
logger = TensorBoardLogger("lightning_logs", name='summary')

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

early_stop_callback = EarlyStopping(monitor='val_loss', patience=5, verbose=False, mode="min")

In [ ]:
trainer = pl.Trainer(check_val_every_n_epoch=1, max_epochs=1, accelerator='gpu',
                     callbacks=[early_stop_callback, checkpoint_callback],
                     logger=logger
                     )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
torch.cuda.empty_cache()
trainer.fit(model, dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 75: 'val_loss' reached 4.30364 (best 4.30364), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
metric = trainer.callback_metrics
loss = metric['val_loss']
float(loss)

4.303642272949219

In [ ]:
def summarizeText(text):
    whitespace_handler = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    text_encoding = tokenizer(
        whitespace_handler(text),
        max_length=400,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    generated_ids = model.model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
text ="""Actor Sushant Singh Rajput has said that he doesn't mind men flirting with him and takes it as a compliment. However,
he added that he doesn't usually expect attention from men. Meanwhile, it has been rumoured that Sushant has been dating actress
Kriti Sanon since they started filming for their upcoming film 'Raabta'. """
print(len(text))
summarizeText(text)

322


"Sushant Singh Rajput has said that he doesn't mind men flirting with him and takes it as a compliment . However, rumoured that the actor has been dating actress Kriti Sanon since they started filming for their upcoming film 'Raabta'"

In [ ]:
torch.save({'model_state_dict': model.state_dict(),
    'loss': loss}, 't5-small.pt')

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
prediction_model = T5smallFinetuner(model=base_model, tokenizer=tokenizer)

In [ ]:
state_dict = torch.load('/content/t5-small.pt')

In [ ]:
prediction_model.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
prediction_model.eval()

In [ ]:
def summarizeTextP(text, model):
    whitespace_handler = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    text_encoding = tokenizer(
        whitespace_handler(text),
        max_length=400,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    generated_ids = model.model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=100,
        num_beams=4,
        no_repeat_ngram_size=2,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
text ="""Actor Sushant Singh Rajput has said that he doesn't mind men flirting with him and takes it as a compliment. However,
he added that he doesn't usually expect attention from men. Meanwhile, it has been rumoured that Sushant has been dating actress
Kriti Sanon since they started filming for their upcoming film 'Raabta'. """
print(len(text))
summarizeTextP(text, model=prediction_model)

322


"Sushant Singh Rajput has said that he doesn't mind men flirting with him and takes it as a compliment . However, it has been rumoured that she is dating actress Kriti Sanon since they started filming for their upcoming film 'Raabta'"